Installation OF Langchain Community   

In [1]:
!pip install langchain_community

Load LLM

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")
llm
response = llm.invoke("Do you know about Claude 3?")
print(response)

Claude 3!

Yes, I'm familiar with Claude 3. Claude is a popular online learning platform that provides interactive coding lessons for kids. The platform has grown in popularity over the years, and Claude 3 is the latest version of their software.

In Claude 3, students can learn to code through a variety of engaging activities, puzzles, and games. The platform focuses on developing programming skills while fostering creativity, problem-solving, and critical thinking. It's designed for kids aged 6-12 (or even older) who are new to coding or looking to improve their coding skills.

What specific aspects of Claude 3 would you like to know more about?


Create Prompt Template

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system","You are an Artificial Intelligence News Reporter."),
    ("user", "{query}")
    ])

Merge Prompt Template & LLM to create Langchain

In [5]:
llm_app = prompt|llm
llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama(model='llama3')

In [6]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})
print(response)

Breaking news, folks! I've got the latest scoop on Claude 3. For those who may not be familiar, Claude 3 is a cutting-edge AI language model developed by Meta AI. It's been making waves in the tech community with its impressive capabilities.

According to reports, Claude 3 has achieved state-of-the-art results on several natural language processing (NLP) benchmarks. This includes tasks such as conversational dialogue, question-answering, and even generating human-like text.

What sets Claude 3 apart from other AI models is its ability to learn and adapt at an incredible pace. It can process vast amounts of data in a short period, making it a powerful tool for applications like customer service chatbots, language translation, and content generation.

I've been following the developments on Claude 3 closely, and I'm excited to report that this AI model has the potential to revolutionize the way we interact with technology. With its advanced capabilities and rapid learning abilities, Clau

Building of RAG Application

Loading External Data

Beautiful Soup is a library that makes it easy to scrape information from web pages. It sits atop an HTML or XML parser, providing Pythonic idioms for iterating, searching, and modifying the parse tree.

In [7]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")
docs  = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
urls=[
    "https://www.anthropic.com/news/releasing-claude-instant-1-2",
    "https://www.anthropic.com/claude"
]

docs = []
for url in urls:
    loader=WebBaseLoader(url)
    docs.extend(loader.load())
len(docs)


2

In [15]:
docs[0]

Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeOverviewTeamAPIPricingResearchCompanyCareersNewsAnnouncementsReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compa

Generate Embeddings

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings_llm = OllamaEmbeddings(model="llama3")
embeddings= embeddings_llm.embed_query("How are you?")
embeddings[:5]

[-1.352015733718872,
 -1.3749171495437622,
 1.9702991247177124,
 -0.39787557721138,
 -3.025041103363037]

In [ ]:
type(embeddings), len(embeddings)
(list, 4096)

In [11]:
embeddings = embeddings_llm.embed_documents([
    "Claude 3 is latest Conversional AI Model from Anthropic",
    "Gemini is latest Conversional AI Model from Google",
    "Llama-3 is latest Conversional AI Model from Meta",
    "Mixtral is latest Conversional AI Model from Mistral AI",
    "GPT-4 is latest Conversional AI Model from OpenAI"
])
len(embeddings),type(embeddings),len(embeddings[0])

(5, list, 4096)

Build Index in Vector Store

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [12]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl (14.6 MB)


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitters = RecursiveCharacterTextSplitter()
documents = text_splitters.split_documents(docs)
len(documents)

2

In [16]:
from langchain_community.vectorstores import FAISS
#documents: List of Documents to add to the vectorstore.
#embedding: Embedding function to use.
vector_index=FAISS.from_documents(documents, embeddings_llm) 
vector_index

3 functions to create index
> from_documents()
> from_embeddings()
> from_texts()

In [18]:
retriever = vector_index.as_retriever()
relevant_docs = retriever.invoke({"input":"Do you know Claude 3?"})
len(relevant_docs)

2

In [19]:
relevant_docs

[Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeOverviewTeamAPIPricingResearchCompanyCareersNewsAnnouncementsReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation comp

In [20]:
for doc in relevant_docs:
    print(f"Title:{doc.metadata['title']}, Source:{doc.metadata['source']}")

Title:Releasing Claude Instant 1.2 \ Anthropic, Source:https://www.anthropic.com/news/releasing-claude-instant-1-2
Title:Meet Claude \ Anthropic, Source:https://www.anthropic.com/claude


Create Retrieval Chain

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based in the provided context and your internal knowledge.
    Give priority to context and if you are not sure then say you are not aware of topic:
    
    <context>
    {context}
    </context>

    Question: {input}
    """)
#document_chain = prompt|llm
document_chain = create_stuff_documents_chain(llm, prompt)
response = document_chain.invoke({
    "input":"Do you know about Claude3?",
    "context":[Document(page_content="Claude 3 is latest Conventional AI Model fom Anthropic")]
})
print(response)

I have been provided with a specific context, which mentions Claude 3 as the latest conventional AI model from Anthropic. Given this context, I am aware of Claude 3 and can confirm that it is indeed the latest Conventional AI Model from Anthropic.


In [22]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000122F2A3DD60>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Answer the following question based in the provided context and your internal knowledge.\n    Give priority to context and if you are not sure then say you are not aware of topic:\n    \n    <context>\n    {context}\n    </context>\n\n    Question: {input}\n    '))])
            | Ollama(

In [24]:
response = retrieval_chain.invoke({"input":"Do you know about Claude 3?"})
type(response)

dict

In [25]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [26]:
print(response["answer"])

I'm not aware of a specific mention or details about Claude 3 in the provided context. However, I can see that there is a mention of Claude 3.5 Sonnet in the original text, which appears to be a model within the Claude family. It seems to offer advanced reasoning capabilities and has been introduced as part of the company's efforts to raise the industry bar for intelligence. If you're looking for more information about Claude 3, I'd suggest reaching out to Anthropic PBC or exploring their website and resources for further details.


In [27]:
for doc in response["context"]:
    print(f"Title: {doc.metadata['title']}, Source:{doc.metadata['source']}")

Title: Meet Claude \ Anthropic, Source:https://www.anthropic.com/claude
Title: Releasing Claude Instant 1.2 \ Anthropic, Source:https://www.anthropic.com/news/releasing-claude-instant-1-2
